In [1]:
from IPython.display import display_html
from sqlalchemy import create_engine
import pandas as pd


host='localhost'
database='eicu'
username='purav'
port = "5432"

postgres_str =('postgresql://'+username+':root'+'@'+host+':'+port+'/'+database)
#   Create the connection
cnx = create_engine(postgres_str)

In [2]:
patient = pd.read_sql('SELECT * FROM eicuu.patient', cnx)
#diagnosis = pd.read_sql('SELECT * FROM diagnosis', con)
apache = pd.read_sql('SELECT * FROM eicuu.apacheapsvar', cnx)

result1 = pd.merge(patient,
                 apache,
                 on="patientunitstayid", 
                 how='inner')

In [ ]:
patient_count = pd.read_sql('SELECT hospitalid ,count(*) FROM eicuu.patient group by hospitalid', cnx)
# vital = pd.read_sql('SELECT * FROM eicuu.vitalPeriodic where patientunitstayid=176895 limit 100', cnx)



In [ ]:
# print(labs.columns)
# print(vital.columns)
# print(vital.patientunitstayid.unique())
print(patient_count.head(50))

In [3]:
result=result1.drop(['patienthealthsystemstayid', 'ethnicity','wardid','apacheadmissiondx','admissionheight','hospitaladmittime24',
        'hospitaladmitoffset','hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation','unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24','meanbp',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid','apacheapsvarid', 'intubated','dialysis','meds', 'hematocrit',
       'creatinine', 'albumin','pco2','glucose'],axis = 1)
print(result.columns)

Index(['patientunitstayid', 'gender', 'age', 'hospitalid',
       'hospitaldischargestatus', 'vent', 'eyes', 'motor', 'verbal', 'urine',
       'wbc', 'temperature', 'respiratoryrate', 'sodium', 'heartrate', 'ph',
       'pao2', 'bun', 'bilirubin', 'fio2'],
      dtype='object')


In [4]:
ids = (result.hospitalid.unique())
print(type(ids[0]))

<class 'numpy.int64'>


In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

X_temp = result
X_temp = X_temp[X_temp.hospitaldischargestatus != '']
# X_temp=X_temp.dropna()

# Mean Imputation for Age
X_temp.replace({'age': {'': '0','> 89': '91'}},inplace=True)
X_temp = X_temp.astype({"age": np.int})
X_temp.replace({'age': {0: int(X_temp.age.mean())}},inplace=True)
# print(X_temp.age.unique())

# Removing age below 18
X_temp = X_temp[X_temp.age >= 18]

# Removing records with no gender given
X_temp = X_temp[X_temp.gender != '']

# gender = pd.DataFrame(X_temp.gender.map({'Male': 'Male', 'Female': 'Female','Other':'Other','Unknown':'Unknown'}))
# print(gender.gender.unique())
# # print(gender.gender.unique())

# X_temp['gender'] = gender['gender']

hds = pd.DataFrame(X_temp['hospitaldischargestatus'])
X_temp.drop(['hospitaldischargestatus'],axis=1,inplace=True)

X_cat = X_temp.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X_temp.select_dtypes(include=['int64', 'float64'])                                                                                                         

X_cat = pd.get_dummies(X_cat)                                                                                                                                 

X_temp=pd.concat([X_cat,X_dog,hds],axis=1)

hds = pd.DataFrame(X_temp.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))
X_temp['hospitaldischargestatus'] = hds['hospitaldischargestatus']

X_alive = X_temp.loc[X_temp['hospitaldischargestatus'] == 1]
X_expired = X_temp.loc[X_temp['hospitaldischargestatus'] == 0]

# X_alive_restricted = X_alive.tail(16000)

# X_temp = X_alive_restricted.append(X_expired)

for i in ids:
    
    print(i)
    X_train = X_temp[X_temp.hospitalid != i]
    X_test = X_temp[X_temp.hospitalid == i]
    
    X_train = pd.DataFrame(X_train.drop(['patientunitstayid'],axis=1))
    X_test = pd.DataFrame(X_test.drop(['patientunitstayid'],axis=1))
    
    y_train = pd.DataFrame(X_train['hospitaldischargestatus'])                
    y_test = pd.DataFrame(X_test['hospitaldischargestatus'])                

    X_train = pd.DataFrame(X_train.drop(['hospitaldischargestatus'],axis=1))
    X_test = pd.DataFrame(X_test.drop(['hospitaldischargestatus'],axis=1))
    
#     X = pd.DataFrame(X_temp.drop(['patientunitstayid'],axis=1))
#     Y = pd.DataFrame(X_temp['hospitaldischargestatus'])                
#     X = pd.DataFrame(X.drop(['hospitaldischargestatus'],axis=1))

#     print(X.columns)

    # # Y = Y[Y.hospitaldischargestatus != '']

    # # df = Y.groupby('hospitaldischargestatus').size()
    print(X_train.shape,X_test.shape)

#     X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, stratify=Y)

    # print(X_train.shape,y_train.shape)
    # print(X_test.shape,y_test.shape)

    log_regressor   = LogisticRegression(penalty='l2', max_iter = 1e8)
    log_regressor.fit(X_train, y_train)

    pred =pd.DataFrame(log_regressor.predict(X_test),columns=["a"])
    log_f1_score=(f1_score(y_test,pred))

    # print(y_test.hospitaldischargestatus.unique())
    # print(pred.groupby('a').size())
    # print(y_test.groupby('hospitaldischargestatus').size())
    # print(confusion_matrix(y_test,pred))
    # print(X_train.columns)
#     print(log_regressor.coef_)

    print(log_f1_score)

59
(168536, 21) (574, 21)


/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9875886524822696
60
(168794, 21) (316, 21)


/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9789983844911146
73
(163135, 21) (5975, 21)


...

/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9718574108818012
196
(168763, 21) (347, 21)


/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9607250755287008
195
(167964, 21) (1146, 21)


/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/purav/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
